**Objetivo**:
- Crear un requerimiento a la API de twitter y agregar parametros a la requisición para poder obtener tweets en idioma castellano y aquellos que se hicieron en la región de España.

# Librerias

In [ ]:
import tweepy  #Se usa la versión 4.12.1

from tweepy import Stream, OAuthHandler
from tweepy.streaming import Stream

import json
import pandas as pd
import numpy as np

# Autenticación de Twitter

In [ ]:
#Twitter app information
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

# API's setup:

In [ ]:
#Get the OAuth token
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True)

# API para solicitar tweets

Primero se hace el resultado de como es un requerimiento normal de la API, solicitando la palabra 'catar' con un total de 200 tweets.



https://docs.tweepy.org/en/stable/api.html#tweepy.API.search_tweets


In [ ]:
# Solicitando a la api que busque la palabra 'catar' con los primeros 200 resultados que nos da resultado un json
results = api.search_tweets(q='catar', count=200)

# Cada tweet da como resutado un archivo json que se concatenara para tenerlo todo en uno
json_data = [r._json for r in results]

# El concatenado de archivos json se guarda en un Pandas DataFrame
df = pd.json_normalize(json_data)

In [ ]:
# Se guarda el nombre de las columnas en una lista para poder inspeccionarlas de mejor manera
col = df.columns.values

In [ ]:
# Acotamos la base a solamente las columans que necesitamos
df1 = df[['created_at','text','user.name','lang']]
df1.head(10)

,created_at,text,user.name,lang
0,Sun Dec 18 21:21:09 +0000 2022,#DirectoGol Sois miserables periodistas y mise...,bernardo sestayo,es
1,Sun Dec 18 21:21:09 +0000 2022,RT @DCM_online: VÍDEO: Buenos Aires vem abaixo...,Leila Maria LULA,pt
2,Sun Dec 18 21:21:09 +0000 2022,RT @Gremio: Alguém duvida que essa foi a cara ...,Maria Luisa,pt
3,Sun Dec 18 21:21:08 +0000 2022,RT @EmmaRincon: .@elonmusk apareció en Catar p...,Agrogenomica,es
4,Sun Dec 18 21:21:05 +0000 2022,RT @elchiringuitotv: 🇦🇷🐐 Messi ha batido... ¡1...,Nahuel Pesquera,es
5,Sun Dec 18 21:21:03 +0000 2022,"RT @fontes_mariana: Alberto Fernández, preside...",hey_ho,pt
6,Sun Dec 18 21:21:02 +0000 2022,"🚨 #AlertaADN\nReportan disturbios en París, Fr...",adn40,es
7,Sun Dec 18 21:21:01 +0000 2022,@ActualidadRT's account has been withheld in B...,Carlos Gomez,es
8,Sun Dec 18 21:21:00 +0000 2022,RT @EspositoBarral: Comando Militar do Leste n...,Rita Ritz,pt
9,Sun Dec 18 21:20:59 +0000 2022,"RT @AFPespanol: #ÚLTIMAHORA Lionel Messi, mejo...",Sebastián,es


# 1. API para solicitar tweets solamente en idioma español

Podemos ver gracias a la documentación que la variable lang tiene estas condiciones:

- **lang**: Restricts tweets to the given language, given by an ISO 639-1 code. Language detection is best-effort.

In [ ]:
# Idiomas obtenidos en nuestra busqueda anterior
df1.lang.value_counts()

es    66
pt    34
Name: lang, dtype: int64

 Ahora hacemos el mismo ejercicio notando el cambio en la variable lang para poder obtener solamente los tweets en idioma español

In [ ]:
# Solicitando a la api que busque la palabra 'catar' con los primeros 200 resultados que nos da resultado un json
results2 = api.search_tweets(q='catar', count=200, lang='es')
# Cada tweet da como resutado un archivo json que se concatenara para tenerlo todo en uno
json_data2 = [r._json for r in results2]
# El concatenado de archivos json se guarda en un Pandas DataFrame
df2 = pd.json_normalize(json_data2)
# Acotamos la base a solamente las columans que necesitamos
df2 = df2[['created_at','text','user.name','lang','geo', 'coordinates', 'place']]
df2.head()

,created_at,text,user.name,lang,geo,coordinates,place
0,Sun Dec 18 21:21:09 +0000 2022,#DirectoGol Sois miserables periodistas y mise...,bernardo sestayo,es,None,None,None
1,Sun Dec 18 21:21:08 +0000 2022,RT @EmmaRincon: .@elonmusk apareció en Catar p...,Agrogenomica,es,None,None,None
2,Sun Dec 18 21:21:05 +0000 2022,RT @elchiringuitotv: 🇦🇷🐐 Messi ha batido... ¡1...,Nahuel Pesquera,es,None,None,None
3,Sun Dec 18 21:21:02 +0000 2022,"🚨 #AlertaADN\nReportan disturbios en París, Fr...",adn40,es,None,None,None
4,Sun Dec 18 21:21:01 +0000 2022,@ActualidadRT's account has been withheld in B...,Carlos Gomez,es,None,None,None


Gracias a la documentación que la variable lang tiene estas condiciones podemos observar que ahora todos nuestros dataframe son obtenidos en idioma español

In [ ]:
# Idiomas obtenidos en nuestra busqueda
df2.lang.value_counts()

es    100
Name: lang, dtype: int64

# 2. API para solicitar tweets por región (España)

Gracias a la documentación obtenemos que:
- **geocode**: Returns tweets by users located within a given radius of the given latitude/longitude. The location is preferentially taking from the Geotagging API, but will fall back to their Twitter profile. The parameter value is specified by “latitude,longitude,radius”, where radius units must be specified as either “mi” (miles) or “km” (kilometers). Note that you cannot use the near operator via the API to geocode arbitrary locations; however you can use this geocode parameter to search near geocodes directly. A maximum of 1,000 distinct “sub-regions” will be considered when using the radius modifier.

Se obtiene la longitud y latitud gracias a la pagina de developers que viene las coordenadas a nivel páis

https://developers.google.com/public-data/docs/canonical/countries_csv

In [ ]:
# Latitud
lat = 40.463667
# Longitud
long= -3.74922

In [ ]:
# Solicitando a la api que busque la palabra 'catar' con los primeros 200 resultados que nos da resultado un json
# Agregamos las coordinadas de España
results = api.search_tweets(q='comida', count=200, lang='es', geocode='40.463667,-3.74922,600km')

# Cada tweet da como resutado un archivo json que se concatenara para tenerlo todo en uno
json_data = [r._json for r in results]

# El concatenado de archivos json se guarda en un Pandas DataFrame
df = pd.json_normalize(json_data)

df.head(5)

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang
0,Sun Dec 18 21:21:00 +0000 2022,1604587559065194497,1604587559065194497,Una verdadera vergüenza la pésima organización...,True,"<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Dec 18 21:20:00 +0000 2022,1604587306161242114,1604587306161242114,me dejé toda la comida de esta semana🥲,False,"<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sun Dec 18 21:19:01 +0000 2022,1604587061264236545,1604587061264236545,"@mariajustcrying así me gusta, churros the bes...",False,"<a href=""http://twitter.com/download/android"" ...",1.604585e+18,1604585306363142144,1.303729e+18,1303729161173970944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sun Dec 18 21:18:59 +0000 2022,1604587050262564867,1604587050262564867,@pericoperez123 @MrClydeBarrow @MPadremanyFont...,True,"<a href=""http://twitter.com/download/android"" ...",1.604581e+18,1604580894718689283,5.467177e+08,546717698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sun Dec 18 21:16:34 +0000 2022,1604586441874571264,1604586441874571264,"@ionebelarra Si vamos tan bien, lo del cheque ...",False,"<a href=""http://twitter.com/download/iphone"" r...",1.604481e+18,1604481224172396544,3.447393e+08,344739325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Idiomas encontrados dentro de los tweets de España

In [ ]:
df['lang'].value_counts()

es    100
Name: lang, dtype: int64